In [2]:
#This script collects all downloaded data, sorts it, and prepares it to be sanitized

import os
import sys
import shutil
import pdb, sys

#https://data.cms.gov/provider-data/topics/hospitals/overall-hospital-quality-star-rating
#https://qualitynet.cms.gov/files/627bb14cb1ccb90016b538a5?filename=OverallStarRating_Jul22_QUS.pdf
#https://qualitynet.cms.gov/files/603966dda413b400224ddf50?filename=Star_Rtngs_CompMthdlgy_v4.1.pdf
#Calculate avg z score for all metrics per group
#Overall stat group ratings for score:
#Mortality 22%
#Safety of Care 22%
#Readmission 22%
#Patient Experience 22%
#Timely and Effective Care 12%
#Re-weight when missing a full measure group!
#End scores are calculated using k-means clustering within the peer groups of facilities who have the same number of stat groups containing at least 3 metrics

#Create list of measure IDs used for CMS overall rating
#7 mortality measures
mortality_measures = [
    "MORT_30_AMI", #30-day death rate for heart attack patients
    "MORT_30_CABG", #Death rate for coronary artery bypass graft surgery patients
    "MORT_30_COPD", #Death rate for chronic obstructive pulmonary disease (COPD) patients
    "MORT_30_HF", #30-day death rate for heart failure patients
    "MORT_30_PN", # 30-day death rate for pneumonia patients
    "MORT_30_STK", # Death rate for stroke patients
    "PSI_4", #Death rate among surgical inpatients with serious treatable complications
    "PSI_4_SURG_COMP"
]

#8 safety measures
safety_measures = [
    "HAI_1_SIR", #Central-line associated bloodstream infection (CLABSI)
    "HAI_1a_SIR", #
    "HAI_2_SIR", #Catheter-associated urinary tract infection (CAUTI)
    "HAI_2a_SIR", #
    "HAI_3_SIR", #Surgical site infection from colon surgery (SSI: Colon)
    "HAI_4_SIR", #Surgical site infection from abdominal hysterectomy (SSI-abdominal hysterectomy)
    "HAI_5_SIR", #Methicillin-resistant Staphylococcus aureus (or MRSA) blood infections (Antibioticresistant blood infections)
    "HAI_6_SIR", # Clostridioides difficile (or C.diff.) infections (Intestinal infections)
    "COMP_HIP_KNEE", # Rate of complications for hip and knee replacement patients
    "PSI_90" #Patient Safety and Adverse Events Composite
]

#11 readmission measures
readmission_measures = [
    "EDAC_30_AMI", #Acute myocardial infarction excess days in acute care (EDAC)
    "READM_30_CABG", # Rate of unplanned readmission after coronary artery bypass graft (CABG) surgery
    "READM_30_COPD", # Rate of unplanned readmission for chronic obstructive pulmonary disease patient
    "EDAC_30_HF", #Heart failure excess days in acute care (EDAC)
    "READM_30_HIP_KNEE", #30-day rate of readmission for hip and knee replacement patients
    "EDAC_30_PN", #Pneumonia excess days in acute care (EDAC)
    "READM_30_HOSP_WIDE", #Rate of readmission after discharge from hospital
    "OP_32", #Facility 7-day risk standardized hospital visit rate after outpatient colonoscopy
    "OP_35_ADM", #Admissions visits for patients receiving outpatient chemotherapy
    "OP_35_ED", #Emergency department (ED) visits for patients receiving outpatient chemotherapy
    "OP_36" # Hospital visits after hospital outpatient surgery
]

#8 patient experience
patient_experience_measures = [
    "H_COMP_1_A_P", #Communication with nurses ALWAYS ++
    "H_COMP_1_U_P", #USUALLY 
    "H_COMP_1_SN_P", #SOMETIMES or NEVER --
    "H_COMP_2_A_P", #Communication with doctors
    "H_COMP_2_U_P",
    "H_COMP_2_SN_P",
    "H_COMP_3_A_P", #Responsiveness of hospital staff
    "H_COMP_3_U_P",
    "H_COMP_3_SN_P",
    "H_COMP_5_A_P", #Communication about medicines
    "H_COMP_5_U_P",
    "H_COMP_5_SN_P",
    "H_COMP_6_Y_P", #YES ++ Discharge information
    "H_COMP_6_N_P",#NO --
    "H_COMP_7_SA", #Care transition
    "H_COMP_7_A",
    "H_COMP_7_D_SD",
    "H_CLEAN_HSP_A_P", #Hospital was clean, averaged with quiet
    "H_CLEAN_HSP_U_P",
    "H_CLEAN_HSP_SN_P",
    "H_QUIET_HSP_A_P", #Hospital was quiet, averaged with clean
    "H_QUIET_HSP_U_P",
    "H_QUIET_HSP_SN_P",
    "H_HSP_RATING_9_10", # rated 9-10 
    "H_HSP_RATING_7_8", # rated 7-8
    "H_HSP_RATING_0_6", # rated 0-6
    "H_RECMND_DY", #definitely yes
    "H_RECMND_PY", #probably yes
    "H_RECMND_DN" #definitely no
]

#12 timely and effective care
timely_and_effective_care_measures = [
    "IMM_3", #Percent of healthcare workers vaccinated against Influenza
    "OP_22", #Percentage of patients who left the emergency department before being seen
    "OP_23", #Percentage of patients who came to the emergency department with stroke symptoms who received brain scan results within 45 minutes of arrival
    "OP_29", #Appropriate follow-up interval for normal colonoscopy in average risk patients
    "OP_33", #External beam radiotherapy for bone metastases
    "PC_01", #: Percent of newborns whose deliveries were scheduled too early (1-3 weeks early), when a scheduled delivery was not medically necessary
    "SEP_1", # Percentage of patients who received appropriate care for severe sepsis and septic shock
    "OP_3b", #Average number of minutes before outpatients with chest pain or possible heart attack who needed specialized care were transferred to another hospital
    #"", #??? not needed / can't find except on website - Avg number of minutes before outpatients with possible heart attack who needed special care were sent to another hospital
    "OP_18b", # Average time patients spent in the emergency department before being sent home
    "OP_8", #Outpatients with low back pain who had an MRI without trying recommended treatments first, such as physical therapy
    "OP_10", #Outpatient CT scans of the abdomen that were “combination” (double) scans
    "OP_13", #Medicare patients who got cardiac imaging stress tests to screen for surgical risk before low-risk outpatient surgery
    "EDV", #???
    "OP_2", #???
    "OP_31", #???
    "PCH_01" #???
]
    
def append_other_spellings(str_list):
    other_spellings = []
    for this_str in str_list:
        if "_" in this_str:
            other_spellings.append(this_str.replace("_","-"))
            other_spellings.append(this_str.replace("_","."))
            other_spellings.append(this_str.replace("_"," "))
            other_spellings.append(this_str.replace("_",""))
    return str_list + other_spellings

all_measures = mortality_measures + safety_measures + readmission_measures + patient_experience_measures + timely_and_effective_care_measures
all_measures = append_other_spellings(all_measures)
    
facility_id_cols = [
    "Facility_Name",
    "Hospital_Name",
    "Provider_Name",
    "VHA_Facility",
    "Facility_ID",
    "Hospital_ID",
    "Provider_ID",
    "CCN#"
]
facility_id_cols = append_other_spellings(facility_id_cols)

measure_id_cols = [
    "Measure_ID",
    "HCAHPS_Measure_ID",
    "TRISS_Measure_ID"
]
measure_id_cols = append_other_spellings(measure_id_cols)

score_cols = [
    "Score", "Rate",
    "Answer_Percent",
    "HCAHPS_Answer_Percent",
    "Observed_Rate_per_1,000",
    "Observed_rate_per_1,000",
    "National Rate"
]
score_cols = append_other_spellings(score_cols)

#remove folders to prevent collision
if os.path.exists("measure_files"):
    shutil.rmtree("measure_files")
if os.path.exists("national_files"):
    shutil.rmtree("national_files")
if os.path.exists("measure_files_trimmed"):
    shutil.rmtree("measure_files_trimmed")
if os.path.exists("junk"):
    shutil.rmtree("junk")
if os.path.exists("VBP"):
    shutil.rmtree("VBP")

#get original directory list then create directories to sort files into
work_dir = os.getcwd()
os.makedirs("measure_files")
os.makedirs("national_files")
os.makedirs("junk")
os.makedirs("VBP")
os.makedirs("junk\\nontable")
os.makedirs("junk\\state_file")
os.makedirs("junk\\no_measure_col")
os.makedirs("junk\\no_score_col")

In [3]:
#prepare count
totalFiles = 0
work_folder = "work_data\\"
good_folder = "measure_files\\"
natl_folder = "national_files\\"
junk_folder = "junk\\"
vbp_folder = "VBP\\"
trimmed_folder = "measure_files_trimmed\\"
folder_list = os.listdir(work_folder)
for folder in folder_list:
    folder = work_folder + folder
    if not os.path.isdir(folder): continue
    totalFiles = totalFiles + len(os.listdir(folder))
    #print(folder + " - " + str(totalFiles))

In [4]:
#get every file in every directory, check for measures, then sort
#moving files is not strictly necessary except for debugging; only measure files are important
fileNum = 0
for fldr in folder_list:
    folder = work_folder + fldr
    #pdb.set_trace()
    if not os.path.isdir(folder): continue
    timestamp = folder[:7]
    file_list = os.listdir(folder)
    folder += "\\"
    
    for file in file_list:
        fileNum += 1
        print("Reading " + folder + file + " [" + str(fileNum) + " of " + str(totalFiles) + "]...")
        
        #If the file is not a .csv, throw it out
        if not file[-4:]==".csv":
            print("\tJunk file - not a table")
            moved_to = shutil.move(folder + file, junk_folder + "nontable\\" + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
            
        #Check if VBP file
        if file[:4]=="hvbp":
            print("\tVBP file")
            moved_to = shutil.move(folder + file, vbp_folder + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
        
        #There are some useless "state" files with no measure data
        if "State" in file or "STATE" in file:
            print("\tJunk file - state file")
            moved_to = shutil.move(folder + file, junk_folder + "state_file\\" + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
        
        #read first line of the file to check for relevant columns
        r = open(folder + file, "r", encoding="utf-8", errors="replace")
        header = r.readline()
        r.close()
        
        #if there's no measure id column, throw out
        if not any(variable in header for variable in measure_id_cols):
            print("\tJunk file - no measure id column")
            moved_to = shutil.move(folder + file, junk_folder + "no_measure_col\\" + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
            
        #Hold off on throwing out score columns until Measure IDs are picked?
        #if there's not score column, throw out
        if not any(variable in header for variable in score_cols):
            print("\tJunk file - no score column")
            moved_to = shutil.move(folder + file, junk_folder + "no_score_col\\" + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
            
        #check for HCAHPS final score?
        
        #check if it's a national measures table, state measures table, or facility measures table
        #national tables list aggregate statistics per measure according to HCAHPS org calculations
        #State tables list measure stats by state, we can't use this data
        #facility measures list measures and stats per facility
        national = False
        if "National" in file or "NATIONAL" in file: national = True
        if not national and not any(variable in header for variable in facility_id_cols):
            #If there are no facility identifying columns, it could be either a state or national file
            if "State" in header:
                print("\tJunk file - state file")
                moved_to = shutil.move(folder + file, junk_folder + "state_file\\" + folder[10:18] + file)
                print("\tMoved to " + moved_to)
                continue
            else: national = True
        if national:
            #separate national files from facility files. National files will be processed more in the future
            #to determine if they contain data on relevant measures
            print("\tNational file - separating from facility files")
            moved_to = shutil.move(folder + file, natl_folder + folder[10:18] + file)
            print("\tMoved to " + moved_to)
            continue
            
        print("\tHospital measure file")
        moved_to = shutil.move(folder + file, good_folder + folder[10:18] + file)
        print("\tMoved to " + moved_to)
        
#shutil.rmtree("work_data")

Reading work_data\2017_04_hos_revised_flatfiles_archive\Ambulatory Surgical Measures-Facility.csv [1 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_04_Ambulatory Surgical Measures-Facility.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Ambulatory Surgical Measures-National.csv [2 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_04_Ambulatory Surgical Measures-National.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Ambulatory Surgical Measures-State.csv [3 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2017_04_Ambulatory Surgical Measures-State.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Complications - Hospital.csv [4 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_04_Complications - Hospital.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Complications - National.csv [5 of 1853]...
	National file - separating from facility files
	Moved 

	Hospital measure file
	Moved to measure_files\2017_04_Medicare Hospital Spending per Patient - Hospital.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Medicare Hospital Spending per Patient - National.csv [53 of 1853]...
	National file - separating from facility files
	Moved to national_files\2017_04_Medicare Hospital Spending per Patient - National.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Medicare Hospital Spending per Patient - State.csv [54 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2017_04_Medicare Hospital Spending per Patient - State.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\MORT_READM_April2017.csv [55 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_04_MORT_READM_April2017.csv
Reading work_data\2017_04_hos_revised_flatfiles_archive\Outpatient Imaging Efficiency - Hospital.csv [56 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_04_Outpatient Imaging Efficiency - Hospital.csv
Read

Reading work_data\2017_07_hos_revised_flatfiles_archive\FY2015_Net_Change_in_Base_Op_DRG_Payment_Amt.csv [93 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_07_FY2015_Net_Change_in_Base_Op_DRG_Payment_Amt.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\FY2015_Percent_Change_in_Medicare_Payments.csv [94 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_07_FY2015_Percent_Change_in_Medicare_Payments.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\FY2015_Value_Based_Incentive_Payment_Amount.csv [95 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_07_FY2015_Value_Based_Incentive_Payment_Amount.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\GLOBAL_July2017_05092017.csv [96 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_07_GLOBAL_July2017_05092017.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\HBIPS_July2017_05092017.csv [97 of 1853

	Moved to junk\state_file\2017_07_Timely and Effective Care - State.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\Value of Care - National.csv [147 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2017_07_Value of Care - National.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\VA_IPSHEP_Jul2017CMS_08MAY17.csv [148 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_07_VA_IPSHEP_Jul2017CMS_08MAY17.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\VA_PSI_July2017_05092017.csv [149 of 1853]...
	Hospital measure file
	Moved to measure_files\2017_07_VA_PSI_July2017_05092017.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\VHA measure dates.csv [150 of 1853]...
	Junk file - no score column
	Moved to junk\no_score_col\2017_07_VHA measure dates.csv
Reading work_data\2017_07_hos_revised_flatfiles_archive\VHA provider level data.csv [151 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_co

	Moved to measure_files\2017_10_Outpatient Imaging Efficiency - Hospital.csv
Reading work_data\2017_10_hos_revised_flatfiles_archive\Outpatient Imaging Efficiency - National.csv [202 of 1853]...
	National file - separating from facility files
	Moved to national_files\2017_10_Outpatient Imaging Efficiency - National.csv
Reading work_data\2017_10_hos_revised_flatfiles_archive\Outpatient Imaging Efficiency - State.csv [203 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2017_10_Outpatient Imaging Efficiency - State.csv
Reading work_data\2017_10_hos_revised_flatfiles_archive\Outpatient Procedures - Volume.csv [204 of 1853]...
	Junk file - no score column
	Moved to junk\no_score_col\2017_10_Outpatient Procedures - Volume.csv
Reading work_data\2017_10_hos_revised_flatfiles_archive\Payment - National.csv [205 of 1853]...
	Junk file - no score column
	Moved to junk\no_score_col\2017_10_Payment - National.csv
Reading work_data\2017_10_hos_revised_flatfiles_archive\Payment - State.

...

	Moved to measure_files\2022_04_Healthcare_Associated_Infections-Hospital.csv
Reading work_data\2022_04_hospitals\Healthcare_Associated_Infections-National.csv [1594 of 1853]...
	National file - separating from facility files
	Moved to national_files\2022_04_Healthcare_Associated_Infections-National.csv
Reading work_data\2022_04_hospitals\Healthcare_Associated_Infections-State.csv [1595 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2022_04_Healthcare_Associated_Infections-State.csv
Reading work_data\2022_04_hospitals\HospitalCompare-DataDictionary.pdf [1596 of 1853]...
	Junk file - not a table
	Moved to junk\nontable\2022_04_HospitalCompare-DataDictionary.pdf
Reading work_data\2022_04_hospitals\Hospitals_CASPER_ASPEN_Contacts.csv [1597 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_04_Hospitals_CASPER_ASPEN_Contacts.csv
Reading work_data\2022_04_hospitals\Hospital_General_Information.csv [1598 of 1853]...
	Junk file - no measure id colu

	Moved to measure_files\2022_04_VA_TE.csv
Reading work_data\2022_04_hospitals\Veterans_Health_Administration_Provider_Level_Data.csv [1640 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_04_Veterans_Health_Administration_Provider_Level_Data.csv
Reading work_data\2022_07_hospitals\ASCQR_OAS_CAHPS_BY_ASC.csv [1641 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_07_ASCQR_OAS_CAHPS_BY_ASC.csv
Reading work_data\2022_07_hospitals\ASCQR_OAS_CAHPS_NATIONAL.csv [1642 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_07_ASCQR_OAS_CAHPS_NATIONAL.csv
Reading work_data\2022_07_hospitals\ASCQR_OAS_CAHPS_STATE.csv [1643 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2022_07_ASCQR_OAS_CAHPS_STATE.csv
Reading work_data\2022_07_hospitals\ASC_Facility.csv [1644 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_07_ASC_Facility.csv
Reading work_data\2022_07_hospital

	Moved to measure_files\2022_07_PCH_HEALTHCARE_ASSOCIATED_INFECTIONS_HOSPITAL.csv
Reading work_data\2022_07_hospitals\PCH_ONCOLOGY_CARE_MEASURES_HOSPITAL.csv [1696 of 1853]...
	Junk file - no score column
	Moved to junk\no_score_col\2022_07_PCH_ONCOLOGY_CARE_MEASURES_HOSPITAL.csv
Reading work_data\2022_07_hospitals\PCH_OUTCOMES_HOSPITAL.csv [1697 of 1853]...
	Hospital measure file
	Moved to measure_files\2022_07_PCH_OUTCOMES_HOSPITAL.csv
Reading work_data\2022_07_hospitals\PCH_OUTCOMES_NATIONAL.csv [1698 of 1853]...
	National file - separating from facility files
	Moved to national_files\2022_07_PCH_OUTCOMES_NATIONAL.csv
Reading work_data\2022_07_hospitals\readme.txt [1699 of 1853]...
	Junk file - not a table
	Moved to junk\nontable\2022_07_readme.txt
Reading work_data\2022_07_hospitals\Timely_and_Effective_Care-Hospital.csv [1700 of 1853]...
	Hospital measure file
	Moved to measure_files\2022_07_Timely_and_Effective_Care-Hospital.csv
Reading work_data\2022_07_hospitals\Timely_and_Effe

	National file - separating from facility files
	Moved to national_files\2022_10_Maternal_Health-National.csv
Reading work_data\2022_10_hospitals\Maternal_Health-State.csv [1748 of 1853]...
	Junk file - state file
	Moved to junk\state_file\2022_10_Maternal_Health-State.csv
Reading work_data\2022_10_hospitals\Measure_Dates.csv [1749 of 1853]...
	Junk file - no score column
	Moved to junk\no_score_col\2022_10_Measure_Dates.csv
Reading work_data\2022_10_hospitals\Medicare_Hospital_Spending_by_Claim.csv [1750 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2022_10_Medicare_Hospital_Spending_by_Claim.csv
Reading work_data\2022_10_hospitals\Medicare_Hospital_Spending_Per_Patient-Hospital.csv [1751 of 1853]...
	Hospital measure file
	Moved to measure_files\2022_10_Medicare_Hospital_Spending_Per_Patient-Hospital.csv
Reading work_data\2022_10_hospitals\Medicare_Hospital_Spending_Per_Patient-National.csv [1752 of 1853]...
	National file - separating from facility file

	Junk file - no measure id column
	Moved to junk\no_measure_col\2023_01_Data_Updates_January_2023.csv
Reading work_data\2023_01_hospitals\Footnote_Crosswalk.csv [1794 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2023_01_Footnote_Crosswalk.csv
Reading work_data\2023_01_hospitals\FY2021_Distribution_of_Net_Change_in_Base_Op_DRG_Payment_Amt.csv [1795 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2023_01_FY2021_Distribution_of_Net_Change_in_Base_Op_DRG_Payment_Amt.csv
Reading work_data\2023_01_hospitals\FY2021_Net_Change_in_Base_Op_DRG_Payment_Amt.csv [1796 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2023_01_FY2021_Net_Change_in_Base_Op_DRG_Payment_Amt.csv
Reading work_data\2023_01_hospitals\FY2021_Percent_Change_in_Medicare_Payments.csv [1797 of 1853]...
	Junk file - no measure id column
	Moved to junk\no_measure_col\2023_01_FY2021_Percent_Change_in_Medicare_Payments.csv
Reading work_data\2023_01_